# Finetune DistilBERT & RoBERTa for Resume Classification

This notebook finetunes both models on `cleaned_resumes.csv` and **saves them properly** for use in Test 2.

**Output directories:**
- `distilbert_resume_level/` - DistilBERT model + tokenizer
- `roberta_resume_level/` - RoBERTa model + tokenizer

In [ ]:
import pandas as pd
import numpy as np
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Load and Prepare Data

In [ ]:
# Load data
df = pd.read_csv("../cleaned_resumes.csv", engine="python")
print(f"Shape: {df.shape}")
print(f"Target distribution:\n{df['experience_level'].value_counts()}")

In [ ]:
# Prepare text field
TARGET_COL = "experience_level"

def clean_value(v):
    if pd.isna(v):
        return ""
    s = str(v)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def row_to_text(row, target_col):
    parts = []
    for col, val in row.items():
        if col == target_col:
            continue
        s = clean_value(val)
        if s:
            parts.append(f"[{col}] {s}")
    return " ".join(parts)

df["text"] = df.apply(lambda r: row_to_text(r, TARGET_COL), axis=1)
print(f"Average text length: {int(df['text'].str.len().mean())} chars")

In [ ]:
# Create label mappings
labels = sorted(df[TARGET_COL].dropna().unique().tolist())
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for l, i in label2id.items()}
df["label"] = df[TARGET_COL].map(label2id)

print(f"Labels: {labels}")
print(f"label2id: {label2id}")

In [ ]:
# Train/test split
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)
print(f"Train: {train_df.shape[0]}, Test: {test_df.shape[0]}")

## Dataset Class

In [ ]:
class ResumeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

In [ ]:
def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro")
    }

## Training Function

In [ ]:
def train_and_save_model(model_name, output_dir, num_epochs=5):
    """
    Train a model and save both weights and tokenizer.
    
    Args:
        model_name: HuggingFace model name (e.g., 'distilbert-base-uncased')
        output_dir: Directory to save the model
        num_epochs: Number of training epochs
    """
    print(f"\n{'='*60}")
    print(f"Training: {model_name}")
    print(f"Output: {output_dir}")
    print(f"{'='*60}")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenize data
    MAX_LEN = 512
    def tokenize(texts):
        return tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=MAX_LEN
        )
    
    train_enc = tokenize(train_df["text"].tolist())
    test_enc = tokenize(test_df["text"].tolist())
    
    # Create datasets
    train_dataset = ResumeDataset(train_enc, train_df["label"].tolist())
    test_dataset = ResumeDataset(test_enc, test_df["label"].tolist())
    
    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id
    )
    
    # Training arguments
    args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=num_epochs,
        logging_steps=50,
        report_to="none"
    )
    
    # Create trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    # Train
    print("\nStarting training...")
    trainer.train()
    
    # IMPORTANT: Save model weights AND tokenizer
    print(f"\nSaving model to {output_dir}...")
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Verify files were saved
    saved_files = os.listdir(output_dir)
    print(f"Saved files: {saved_files}")
    
    # Check for model weights
    has_weights = any(f.endswith('.bin') or f.endswith('.safetensors') for f in saved_files)
    if has_weights:
        print("Model weights saved successfully!")
    else:
        print("WARNING: Model weights may not have been saved!")
    
    # Final evaluation
    print("\nFinal evaluation on test set:")
    results = trainer.evaluate()
    print(f"Accuracy: {results['eval_accuracy']:.4f}")
    print(f"F1 Macro: {results['eval_f1_macro']:.4f}")
    
    return trainer, results

---
## Train DistilBERT

In [ ]:
distilbert_trainer, distilbert_results = train_and_save_model(
    model_name="distilbert-base-uncased",
    output_dir="distilbert_resume_level",
    num_epochs=5
)

---
## Train RoBERTa

In [ ]:
roberta_trainer, roberta_results = train_and_save_model(
    model_name="roberta-base",
    output_dir="roberta_resume_level",
    num_epochs=5
)

---
## Verify Models Can Be Loaded

In [ ]:
print("Verifying models can be loaded...\n")

for name, path in [("DistilBERT", "distilbert_resume_level"), ("RoBERTa", "roberta_resume_level")]:
    try:
        tokenizer = AutoTokenizer.from_pretrained(path)
        model = AutoModelForSequenceClassification.from_pretrained(path)
        print(f"{name}: Loaded successfully!")
        print(f"  - Config: {model.config.id2label}")
    except Exception as e:
        print(f"{name}: FAILED to load - {e}")

---
## Summary

In [ ]:
print("="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"\nDistilBERT:")
print(f"  Accuracy: {distilbert_results['eval_accuracy']:.4f}")
print(f"  F1 Macro: {distilbert_results['eval_f1_macro']:.4f}")
print(f"  Saved to: distilbert_resume_level/")

print(f"\nRoBERTa:")
print(f"  Accuracy: {roberta_results['eval_accuracy']:.4f}")
print(f"  F1 Macro: {roberta_results['eval_f1_macro']:.4f}")
print(f"  Saved to: roberta_resume_level/")

print("\nModels are ready for Test 2!")